In [ ]:
flex_positions = ["RB", "WR", "TE"]

# Constraint für Flex-Position
prob += pulp.lpSum([player_vars[i] for i in test if df_optimization.loc[i, 'position_group'] in flex_positions]) >= position_requirements["FLEX"]

In [ ]:
def optimize_team(all_players, df_optimization):
    position_requirements = {
        'QB': 1,
        'RB': 2,
        'WR': 2,
        'TE': 1,
    }
    total_players = 6

    # Initialisiere das Problem
    prob = pulp.LpProblem("FantasyFootballProblem", pulp.LpMaximize)

    # Erstelle eine Liste der Spieler (in diesem Fall verwende ich die player_id)
    players = all_players

    # Erstelle ein Wörterbuch mit den Vorhersagen
    predicted_points = df_optimization.loc[players, 'predicted_points'].to_dict()

    # Erstelle ein Wörterbuch mit den Positionen
    positions = df_optimization.loc[players, 'position_group'].to_dict()

    # Erstelle die Variablen für das Problem
    player_vars = pulp.LpVariable.dicts("Player", [player.replace('Player_', '') for player in players], 0, 1, cat='Integer')

    # Zielfunktion: Maximiere die Gesamtpunktzahl
    prob += pulp.lpSum([player_vars[i] * predicted_points[i] for i in players])

    # Constraints für Positionen
    for pos in set(positions.values()):
        prob += pulp.lpSum([player_vars[i] for i in players if positions[i] == pos]) == position_requirements[pos]

    # Constraints für Gesamtzahl der ausgewählten Spieler
    prob += pulp.lpSum([player_vars[i] for i in players]) == total_players

    # Löse das Problem
    prob.solve()

    # Erstelle eine Liste der ausgewählten Spieler
    chosen_players = [v.name.replace('Player_', '') for v in prob.variables() if v.varValue == 1]

    # Gib die Liste der ausgewählten Spieler zurück
    return chosen_players

In [ ]:
chosen_players = []

all_players = df_optimization.index.tolist()

position_counts = {pos: 0 for pos in position_requirements.keys()}

total_rounds = 60
def pick_player(player_list, position_counts):
    for player in player_list:
        if position_counts[df_optimization.loc[player, 'position_group']] < position_requirements[df_optimization.loc[player, 'position_group']]:
            return player
    return None

for round in range(total_rounds):
    # Führen Sie die Optimierung für die verfügbaren Spieler durch
    test = [player for player in all_players if player not in chosen_players]
    best_players = optimize_team(test, df_optimization)  # Ihre Optimierungsfunktion hier
    
    # Wählen Sie Ihren Spieler
    my_player = best_players[0]  # Nehmen wir an, wir wählen immer den besten Spieler


    
    chosen_players.append(my_player)
    position_counts[df_optimization.loc[my_player, 'position_group']] += 1
    
    # Entfernen Sie diesen Spieler aus der Liste der verfügbaren Spieler
    all_players.remove(my_player)
    
    # Wiederholen Sie den Vorgang für die anderen Spieler
    for other_player in range(num_other_players):
        their_player = pick_player(best_players, position_counts)
        if their_player is not None:
            chosen_players.append(their_player)
            position_counts[df_optimization.loc[their_player, 'position_group']] += 1
            all_players.remove(their_player)


In [ ]:
# Vor dem Optimierungsproblem:
opponent_picks = {}  # Dieses Dictionary wird die Picks jedes Managers in jeder Woche speichern
weeks = all_weeks
position = player_data['position_group'].unique().tolist()
for manager in managers:
    if manager != 'Manager1':  # Wir betrachten nur die gegnerischen Manager
        for week in weeks:
            # Wählen Sie die Spieler mit den höchsten erwarteten Punkten in jeder Position aus
            qb_pick = max([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'QB' and (i, week) in predicted_points_per_strength], key=lambda x: x[1])[0]
            wr_picks = sorted([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'WR' and (i, week) in predicted_points_per_strength], key=lambda x: x[1], reverse=True)[:2]  # Die besten zwei WRs
            rb_picks = sorted([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'RB' and (i, week) in predicted_points_per_strength], key=lambda x: x[1], reverse=True)[:2]  # Die besten zwei RBs
            te_pick = max([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'TE' and (i, week) in predicted_points_per_strength], key=lambda x: x[1])[0]
            
            # Speichern Sie die Picks
            opponent_picks[(manager, week)] = [qb_pick] + [pick[0] for pick in wr_picks] + [pick[0] for pick in rb_picks] + [te_pick]

# Berechnen Sie die erforderlichen Punkte, um den gegnerischen Manager zu schlagen
required_points_to_beat_opponent = {}
predicted_points_per_strength = predicted_points_per_strength.to_dict()

for week in weeks:
    required_points_to_beat_opponent[week] = sum(predicted_points_per_strength[(i, week)] for manager in managers if manager != 'Manager1' for i in opponent_picks[(manager, week)])